In [1]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor

In [2]:
data = io.load('data/gauss4')
# data = io.load('data/simulations')

In [3]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

In [10]:
@debug.capture()
class Monitor(Filter):
    def __init__(self, monitor=None, **kwargs):
        super().__init__(**kwargs)
        self._monitored = []
        self.monitor = monitor

    def __call__(self, *args):
        return self._disabled or self._func(self._monitored[0], *args)

    @debug.capture()
    def _exec(self, change):
        print('exec', change)
        self()

    @property
    def monitor(self):
        return self._monitored

    @monitor.setter
    def monitor(self, obj):
        for m in self._monitored:
            # m.unregister(self._exec)
            m.unobserve(self._exec, names='changed')
        self._monitored = [obj] if not isinstance(obj, list) else obj
        for m in self._monitored:
            # m.register(self._exec)
            m.observe(self._exec, names='changed')

### Helper functions

#### view

In [11]:
def update_view(view):
    def _inner(f):
        view.show = filter_tree(view.tree, f)
    return _inner 

In [12]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [13]:
def reduce_view(widget, tree):
    def _inner(f):
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [14]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [15]:
tw = TreeWidget(data.tree)

In [16]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [17]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

In [11]:
f2.func = lambda x, v: v > x

In [12]:
v2.attr = 'parent_fitness'

In [13]:
f2.attr = 'span'
f2.func = lambda x, v: v <=x

### Reduce tree

In [29]:
v3, r3, p3 = reduce(tw, attr='span', title='reduce')

In [31]:
v3.attr = 'parent_fitness'

### fitness

#### test

In [15]:
tw2 = TreeWidget(data.tree)

In [16]:
f4 = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [17]:
def test(f):
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [18]:
m4 = Monitor(f4, func=test)

In [19]:
v4 = TreeView(tw2, attr='span') 

In [20]:
p4 = SidePanel(title='Test')
with p4:
    display(v4, f4)

In [21]:
fitness = view(tw2, attr='fitness', title='fitness')

In [22]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

### Data widget

In [25]:
dw = DataWidget(data=data)

In [26]:
details = DetailsView(data=dw)

In [27]:
p7 = SidePanel(title='details')
with p7:
    display(details)

In [28]:
widgets.dlink((fitness[0], 'details'), (details, 'show'))